In [ ]:
import os, shutil, random

original_dir = r"C:\Users\user\.keras\datasets\flowers"
base_dir = r"C:\Users\user\.keras\datasets\flower_data"

os.makedirs(os.path.join(base_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'validation'), exist_ok=True)

for category in os.listdir(original_dir):
    cat_dir = os.path.join(original_dir, category)
    if os.path.isdir(cat_dir):
        images = os.listdir(cat_dir)
        random.shuffle(images)
        split = int(0.8 * len(images))
        train_imgs = images[:split]
        val_imgs = images[split:]

        os.makedirs(os.path.join(base_dir, 'train', category), exist_ok=True)
        os.makedirs(os.path.join(base_dir, 'validation', category), exist_ok=True)

        for img in train_imgs:
            shutil.copy(os.path.join(cat_dir, img), os.path.join(base_dir, 'train', category, img))
        for img in val_imgs:
            shutil.copy(os.path.join(cat_dir, img), os.path.join(base_dir, 'validation', category, img))


In [ ]:
#Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

#Data Preparation + Augmentation
train_dir = r"C:\Users\user\.keras\datasets\flower_data\train"
val_dir = r"C:\Users\user\.keras\datasets\flower_data\validation"

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,
    shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,target_size=(150,150),batch_size=32,class_mode='categorical')

val_gen = val_datagen.flow_from_directory(val_dir,target_size=(150,150),batch_size=32,class_mode='categorical')

#Build Model (Transfer Learning: VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

for layer in base_model.layers[:-4]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

#Train Model
history = model.fit(train_gen,validation_data=val_gen,epochs=10)

#Evaluate & Confusion Matrix
val_preds = model.predict(val_gen)
y_pred = np.argmax(val_preds, axis=1)
y_true = val_gen.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',xticklabels=val_gen.class_indices.keys(),
            yticklabels=val_gen.class_indices.keys())
plt.title("Confusion Matrix - Flower Classification")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print(classification_report(y_true, y_pred, target_names=val_gen.class_indices.keys()))

#Save Model & Results
model.save("flower_classifier_vgg16.h5")

with open("results.txt", "w") as f:
    acc = history.history['accuracy'][-1]
    val_acc = history.history['val_accuracy'][-1]
    f.write(f"Train Accuracy: {acc:.3f}\nValidation Accuracy: {val_acc:.3f}\n")
